## Design a product

The design of a product is the process of defining characteristics to meet others.

Typically, geometries will be computed to reach physical specifications.

In [75]:
def print_features(cpu):
    print(
        f"With an ambiant temperature of {cpu.fan.T_air}°C:\n"
        f"CPU default features:\n"
        f"\tUsage: {cpu.cpu.usage} %\n"
        f"\tPower: {cpu.cpu.power} W \n"
        f"\tTemperature: {cpu.T_cpu}°C \n"
        f"Exchanger default features:\n"
        f"\tSurface: {cpu.exchanger.surface:.5f} m^2 \n"
    )

### Definition of the specs

Create a new CPU system

In [76]:
from cpu.systems import CPUSystem

# create a system model
cpu = CPUSystem("cpu")


Exchanger surface is designed to target a maximum CPU temperature of 80°C, when air temperature is 40°C and CPU is used at 100%.

In [77]:
cpu.T_cpu = 80.0
cpu.fan.T_air = 40.0
cpu.cpu.usage = 100.0

In [ ]:
from cosapp.drivers import NonLinearSolver

# add a design driver and its definition
design = cpu.add_driver(NonLinearSolver("design"))
design.extend(cpu.design_methods["exchanger_surface"])

Unknowns [1]
  exchanger.surface = 0.01
Equations [1]
  cpu.heat_flow == cpu.power := -20.0

In [79]:
# run design
cpu.run_drivers()
print("Target heat exchanger surface found to comply the imposed problem is: \n")
print_features(cpu)

Target heat exchanger surface found to comply the imposed problem is: 

With an ambiant temperature of 40.0°C:
CPU default features:
	Usage: 100.0 %
	Power: 105.0 W 
	Temperature: 80.0°C 
Exchanger default features:
	Surface: 0.02386 m^2 



### New design can be done instantly

In [80]:
# change air temperature value
cpu.fan.T_air = 30.0

# run again design
cpu.run_drivers()
print_features(cpu)

With an ambiant temperature of 30.0°C:
CPU default features:
	Usage: 100.0 %
	Power: 105.0 W 
	Temperature: 80.0°C 
Exchanger default features:
	Surface: 0.01909 m^2 



### More direct way to run the same design

In [81]:
# clear and define new drivers
cpu.drivers.clear()
design = cpu.add_driver(NonLinearSolver("solver"))

# add a single case driver to set values
runner = design.add_driver(RunSingleCase("runner"))

# design method
design.extend(cpu.design_methods["exchanger_surface"])

# set initial values
design.runner.set_values({"fan.T_air": 40.0, "T_cpu": 80.0, "cpu.usage": 100.0})

### Save the designed system

In [82]:
from cosapp.utils.json import EncodingMetadata
from pathlib import Path

Path("data").mkdir(exist_ok=True)
cpu_json = cpu.save("data/cpu_ref.json", encoding_metadata=EncodingMetadata(with_drivers=False))